In [48]:
import socket
import json


host = "localhost"
port = 8080

def send_and_recieve(json_data):

    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client.connect((host,port))
    login_info = json_data
    login_info = json.dumps(login_info)
    try:
        client.send(bytes(login_info, encoding="ascii"))
        response = client.recv(4096)
        response = response.decode("ascii")
        response = json.loads(response)
    except:
        print("Server returned nothing or request was out of time")
        return ""
    return response

# Testing "login" and action validation

In [85]:
assert send_and_recieve({"malformed": "request"}) == {'state': 'error', 'why': 'badRequest'} # request without fields "name", "action", "actionSpecs"
assert send_and_recieve({'name': "userNotLoggable", "action": "listRooms", "actionSpecs": {}}) == {'state': 'error', 'why': 'userNotAllowedError'} # unauthorized user
assert send_and_recieve({'name': "riccardo", "action": "unknownAction", "actionSpecs": {}}) == {"state": "error", "why": "unknownAction"} # authorized user but unknown action

# testing room create / destroy

In [56]:
assert send_and_recieve({'name': "riccardo", "action": "createRoom", "actionSpecs": {"room": "room1"}}) == {"state": "ok", "info": "roomCreated"}  # create room ok
assert send_and_recieve({'name': "alessandro", "action": "destroyRoom", "actionSpecs": {"room": "room1"}}) == {"state": "error", "why": "notAllowed"} # other user cannot destroy room
assert send_and_recieve({'name': "riccardo", "action": "destroyRoom", "actionSpecs": {"room": "randomRoom"}}) == {"state": "error", "why": "notAllowed"} # cannot destory non-existent room
assert send_and_recieve({'name': "riccardo", "action": "destroyRoom", "actionSpecs": {"room": "room1"}}) == {"state": "ok", "info": "roomDestroyed"} # ok room destroyed
assert send_and_recieve({'name': "riccardo", "action": "destroyRoom", "actionSpecs": {"room": "room1"}}) == {"state": "error", "why": "notAllowed"} # double destroy, impossible


# testing room list

In [79]:
assert send_and_recieve({'name': "riccardo", "action": "listRooms", "actionSpecs": {}}) == {"state": "ok", "roomNames": "[]"}
send_and_recieve({'name': "riccardo", "action": "createRoom", "actionSpecs": {"room": "room1"}})
assert "room1" in send_and_recieve({'name': "riccardo", "action": "listRooms", "actionSpecs": {}})["roomNames"]
send_and_recieve({'name': "riccardo", "action": "createRoom", "actionSpecs": {"room": "room2"}})
assert "room1" in send_and_recieve({'name': "riccardo", "action": "listRooms", "actionSpecs": {}})["roomNames"]
assert "room2" in send_and_recieve({'name': "riccardo", "action": "listRooms", "actionSpecs": {}})["roomNames"]
send_and_recieve({'name': "riccardo", "action": "destroyRoom", "actionSpecs": {"room": "room1"}})
assert "room1" not in send_and_recieve({'name': "riccardo", "action": "listRooms", "actionSpecs": {}})["roomNames"]
assert "room2" in send_and_recieve({'name': "riccardo", "action": "listRooms", "actionSpecs": {}})["roomNames"]
send_and_recieve({'name': "riccardo", "action": "destroyRoom", "actionSpecs": {"room": "room2"}})
assert send_and_recieve({'name': "riccardo", "action": "listRooms", "actionSpecs": {}}) == {"state": "ok", "roomNames": "[]"}

# testing room join

In [84]:
send_and_recieve({'name': "riccardo", "action": "createRoom", "actionSpecs": {"room": "room1"}})
assert send_and_recieve({'name': "riccardo", "action": "joinRoom", "actionSpecs": {"room": "room1"}}) == {'info': 'joinedRoom', 'state': 'ok'}
assert send_and_recieve({'name': "alessandro", "action": "joinRoom", "actionSpecs": {"room": "room1"}}) == {'info': 'joinedRoom', 'state': 'ok'}
send_and_recieve({'name': "riccardo", "action": "sendMessage", "actionSpecs": {"room": "room1", "message": "ciao"}})


{'info': 'messageSent', 'state': 'ok'}